In [1]:
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, vocab_size, embed_dim, max_len=100):
        super().__init__()
        self.token_emb = layers.Embedding(vocab_size, embed_dim)
        self.pos_emb = layers.Embedding(max_len, embed_dim)

    def call(self, x):
        length = tf.shape(x)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        return self.token_emb(x) + self.pos_emb(positions)

In [6]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(embed_dim),
        ])
        self.norm1 = layers.LayerNormalization()
        self.norm2 = layers.LayerNormalization()

    def call(self, x):
        attn_output = self.att(x, x)
        x = self.norm1(x + attn_output)
        ffn_output = self.ffn(x)
        return self.norm2(x + ffn_output)

In [7]:
def build_transformer(vocab_size, max_len, embed_dim, num_heads, ff_dim):
    inputs = layers.Input(shape=(None,))

    x = PositionalEmbedding(vocab_size, embed_dim, max_len)(inputs)
    x = TransformerEncoder(embed_dim, num_heads, ff_dim)(x)

    outputs = layers.Dense(vocab_size, activation='softmax')(x)

    return tf.keras.Model(inputs, outputs)

In [8]:
model = build_transformer(vocab_size=10000,
                          max_len=100,
                          embed_dim=64,
                          num_heads=2,
                          ff_dim=128
                         )

In [9]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ positional_embedding_1               │ (None, None, 64)            │         646,400 │
│ (PositionalEmbedding)                │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_1                │ (None, None, 64)            │          50,048 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, None, 10000)         │         650,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,346,448 (5.14 MB)

 Trainable params: 1,346,448 (5.14 MB)

 Non-trainable params: 0 (0.00 B)

### Implementing a pre-trained Transformer model for text classification.

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
transformer = TFAutoModel.from_pretrained(model_name)

In [ ]:
text = ["I love this movie"]

inputs = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors="tf"
)

In [ ]:
outputs = transformer(inputs)

In [ ]:
cls_embedding = outputs.last_hidden_state[:, 0, :]

In [ ]:
logits = tf.keras.layers.Dense(2)(cls_embedding)